<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=0e3d72fe0381b83b1be729824e4eb7b13fa9d1bdd9967f55cccdd1cf143338c2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=9c5a5ed0c312f60e3e17c26a8ec0543038dd6f3682ff0360fafcb9e3803aa33b
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

'timestamp'
銘柄コード[1954]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9810]の株価情報を取得できなかったため、スキップしました。
経過時間: 191.41738605499268秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1825600,669.200012
1,タマホーム,1419,建設業,500300,3610.000000
2,ＩＮＰＥＸ,1605,鉱業,7807400,1680.500000
3,安藤・間,1719,建設業,825500,1042.000000
4,コムシスホールディングス,1721,建設業,728700,2737.500000
...,...,...,...,...,...
324,乃村工藝社,9716,サービス業,968900,924.000000
325,ＳＣＳＫ,9719,情報・通信業,507800,2233.000000
326,ヤマダホールディングス,9831,小売業,2863900,427.200012
327,モリト,9837,卸売業,523500,1089.000000


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1825600,669.200012
1,ＩＮＰＥＸ,1605,鉱業,7807400,1680.500000
2,安藤・間,1719,建設業,825500,1042.000000
3,コムシスホールディングス,1721,建設業,728700,2737.500000
4,大林組,1802,建設業,2590400,1232.000000
...,...,...,...,...,...
235,エイチ・アイ・エス,9603,サービス業,811700,2022.000000
236,エヌ・ティ・ティ・データ,9613,情報・通信業,3694400,1940.000000
237,乃村工藝社,9716,サービス業,968900,924.000000
238,ＳＣＳＫ,9719,情報・通信業,507800,2233.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1825600,669.200012
1,ＩＮＰＥＸ,1605,鉱業,7807400,1680.500000
2,大林組,1802,建設業,2590400,1232.000000
3,長谷工コーポレーション,1808,建設業,839800,1779.000000
4,ＬＩＦＵＬＬ,2120,サービス業,968200,300.000000
...,...,...,...,...,...
77,商船三井,9104,海運業,5148400,3576.000000
78,川崎汽船,9107,海運業,15441400,3960.000000
79,ＳＧホールディングス,9143,陸運業,1061200,2110.500000
80,ソフトバンク,9434,情報・通信業,5805500,1538.000000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,タマホーム,1419,建設業,500300,3610.000000
1,三井住友建設,1821,建設業,654800,368.000000
2,パーソルホールディングス,2181,サービス業,607900,2678.000000
3,明治ホールディングス,2269,食料品,894600,3315.000000
4,日本駐車場開発,2353,不動産業,3444600,213.000000
...,...,...,...,...,...
84,電源開発,9513,電気・ガス業,702900,2131.500000
85,レノバ,9519,電気・ガス業,792800,1508.000000
86,大阪瓦斯,9532,電気・ガス業,1444700,2134.000000
87,ヤマダホールディングス,9831,小売業,2863900,427.200012


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,三井住友建設,1821,建設業,654800,368.000000
1,日本駐車場開発,2353,不動産業,3444600,213.000000
2,ベネフィット・ワン,2412,サービス業,580000,1416.000000
3,エムスリー,2413,サービス業,3571800,3227.000000
4,博報堂ＤＹホールディングス,2433,サービス業,803500,1526.000000
5,北の達人コーポレーション,2930,化学,1403600,282.000000
6,ペッパーフードサービス,3053,小売業,689800,122.000000
7,ＭｏｎｏｔａＲＯ,3064,小売業,924900,1764.000000
8,ＺＯＺＯ,3092,小売業,1085500,2853.000000
9,クリエイト・レストランツ・ホールディングス,3387,小売業,840400,991.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[1764]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1793]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2754]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3184]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3352]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3437]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3583]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3634]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3847]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3944]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4341]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4645]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5279]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5395]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5458]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5695]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,2726500,2906.0
1,インターライフホールディングス,1418,建設業,853900,216.0
2,中外鉱業,1491,非鉄金属,753800,30.0
3,住石ホールディングス,1514,鉱業,568900,349.0
4,中小企業ホールディングス,1757,建設業,832800,39.0
...,...,...,...,...,...
89,昭文社ホールディングス,9475,情報・通信業,247200,304.0
90,アゴーラ ホスピタリティー グループ,9704,サービス業,1367600,25.0
91,コックス,9876,小売業,218700,164.0
92,小僧寿し,9973,小売業,297600,20.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,インターライフホールディングス,1418,建設業,853900,216.000000
1,住石ホールディングス,1514,鉱業,568900,349.000000
2,中小企業ホールディングス,1757,建設業,832800,39.000000
3,燦キャピタルマネージメント,2134,サービス業,1015200,22.000000
4,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1121800,58.000000
...,...,...,...,...,...
62,アコム,8572,その他金融業,1750300,341.799988
63,トレイダーズホールディングス,8704,証券、商品先物取引業,648900,626.000000
64,昭文社ホールディングス,9475,情報・通信業,247200,304.000000
65,アゴーラ ホスピタリティー グループ,9704,サービス業,1367600,25.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ベクター,2656,小売業,200000,356.0
1,シー・ヴイ・エス・ベイエリア,2687,小売業,2657000,798.0
2,ピクセルカンパニーズ,2743,卸売業,1736700,53.0
3,コメ兵ホールディングス,2780,小売業,335100,4475.0
4,アクサスホールディングス,3536,小売業,240300,139.0
5,クラウディアホールディングス,3607,繊維製品,520000,788.0
6,ブロードバンドタワー,3776,情報・通信業,522600,151.0
7,セプテーニ・ホールディングス,4293,サービス業,354000,418.0
8,Ｓｐｅｅｅ,4499,情報・通信業,354400,2726.0
9,環境管理センター,4657,サービス業,246600,527.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,2726500,2906.0
1,中外鉱業,1491,非鉄金属,753800,30.0
2,アジアゲートホールディングス,1783,建設業,426900,53.0
3,クシム,2345,情報・通信業,449100,323.0
4,セリア,2782,小売業,295400,2389.0
5,石垣食品,2901,食料品,417500,162.0
6,ｅｎｉｓｈ,3667,情報・通信業,631800,333.0
7,リミックスポイント,3825,情報・通信業,481200,225.0
8,エス・サイエンス,5721,非鉄金属,448100,24.0
9,ＥＲＩホールディングス,6083,サービス業,468300,1736.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,中外鉱業,1491,非鉄金属,753800,30.0
1,アジアゲートホールディングス,1783,建設業,426900,53.0
2,クシム,2345,情報・通信業,449100,323.0
3,石垣食品,2901,食料品,417500,162.0
4,ｅｎｉｓｈ,3667,情報・通信業,631800,333.0
5,リミックスポイント,3825,情報・通信業,481200,225.0
6,エス・サイエンス,5721,非鉄金属,448100,24.0
7,高松機械工業,6155,機械,252600,559.0
8,千代田化工建設,6366,建設業,489500,349.0
9,ＧａｍｅＷｉｔｈ,6552,サービス業,398800,330.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3986]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4450]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,228200,919.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,585000,745.0
2,ジーエヌアイグループ,2160,医薬品,827700,1207.0
3,メディネット,2370,サービス業,288700,65.0
4,ウェッジホールディングス,2388,その他金融業,8651200,155.0
...,...,...,...,...,...
99,関通,9326,倉庫・運輸関連業,216500,531.0
100,ｉｓｐａｃｅ,9348,サービス業,258800,1471.0
101,リニューアブル・ジャパン,9522,電気・ガス業,1806300,863.0
102,Ｍ＆Ａ総研ホールディングス,9552,サービス業,1456400,3975.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,585000,745.0
1,ジーエヌアイグループ,2160,医薬品,827700,1207.0
2,フルッタフルッタ,2586,食料品,1730800,79.0
3,ベースフード,2936,食料品,408800,463.0
4,海帆,3133,小売業,2402100,560.0
5,ティーケーピー,3479,不動産業,1214600,2876.0
6,ＧＡ ｔｅｃｈｎｏｌｏｇｉｅｓ,3491,不動産業,329900,1309.0
7,霞ヶ関キャピタル,3498,不動産業,261400,4740.0
8,リネットジャパングループ,3556,小売業,227200,598.0
9,ロコンド,3558,小売業,311900,1629.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ティーケーピー,3479,不動産業,1214600,2876.0
1,霞ヶ関キャピタル,3498,不動産業,261400,4740.0
2,リネットジャパングループ,3556,小売業,227200,598.0
3,モブキャストホールディングス,3664,情報・通信業,750600,91.0
4,ソーシャルワイヤー,3929,情報・通信業,382500,399.0
5,ＷＡＣＵＬ,4173,情報・通信業,1031200,930.0
6,ＨＥＮＮＧＥ,4475,情報・通信業,631800,929.0
7,スペースマーケット,4487,情報・通信業,2612900,463.0
8,免疫生物研究所,4570,医薬品,3512100,585.0
9,カルナバイオサイエンス,4572,医薬品,3127700,901.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,228200,919.0
1,メディネット,2370,サービス業,288700,65.0
2,ウェッジホールディングス,2388,その他金融業,8651200,155.0
3,ベースフード,2936,食料品,408800,463.0
4,サンクゼール,2937,食料品,533500,2737.0
5,農業総合研究所,3541,卸売業,340400,371.0
6,ＦＨＴホールディングス,3777,情報・通信業,4956400,36.0
7,ディー・ディー・エス,3782,情報・通信業,5959500,12.0
8,Ａｉｍｉｎｇ,3911,情報・通信業,1630900,465.0
9,ココナラ,4176,情報・通信業,855800,356.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,メディネット,2370,サービス業,288700,65.0
1,サンクゼール,2937,食料品,533500,2737.0
2,農業総合研究所,3541,卸売業,340400,371.0
3,ＦＨＴホールディングス,3777,情報・通信業,4956400,36.0
4,ディー・ディー・エス,3782,情報・通信業,5959500,12.0
5,ココナラ,4176,情報・通信業,855800,356.0
6,アクリート,4395,情報・通信業,223400,1370.0
7,スマレジ,4431,情報・通信業,212600,2148.0
8,ＢＡＳＥ,4477,情報・通信業,1647100,258.0
9,フリー,4478,情報・通信業,495200,3220.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[1764]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1793]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2754]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3184]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3352]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3437]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3583]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3634]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3847]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3944]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4341]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4645]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5279]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5395]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5458]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5695]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,2726500,2906.000000
1,インターライフホールディングス,1418,建設業,853900,216.000000
2,中外鉱業,1491,非鉄金属,753800,30.000000
3,住石ホールディングス,1514,鉱業,568900,349.000000
4,中小企業ホールディングス,1757,建設業,832800,39.000000
5,燦キャピタルマネージメント,2134,サービス業,1015200,22.000000
6,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1121800,58.000000
7,アルバイトタイムス,2341,サービス業,2866800,146.000000
8,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,4632200,684.000000
9,出前館,2484,情報・通信業,7234500,536.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,2726500,2906.000000
1,インターライフホールディングス,1418,建設業,853900,216.000000
3,住石ホールディングス,1514,鉱業,568900,349.000000
4,中小企業ホールディングス,1757,建設業,832800,39.000000
5,燦キャピタルマネージメント,2134,サービス業,1015200,22.000000
6,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1121800,58.000000
7,アルバイトタイムス,2341,サービス業,2866800,146.000000
8,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,4632200,684.000000
9,出前館,2484,情報・通信業,7234500,536.000000
12,白鳩,3192,小売業,638700,373.000000


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3986]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4450]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,585000,745.0
1,ジーエヌアイグループ,2160,医薬品,827700,1207.0
2,ウェッジホールディングス,2388,その他金融業,8651200,155.0
3,フルッタフルッタ,2586,食料品,1730800,79.0
4,サンクゼール,2937,食料品,533500,2737.0
5,海帆,3133,小売業,2402100,560.0
6,ティーケーピー,3479,不動産業,1214600,2876.0
7,モブキャストホールディングス,3664,情報・通信業,750600,91.0
8,ＦＨＴホールディングス,3777,情報・通信業,4956400,36.0
9,ディー・ディー・エス,3782,情報・通信業,5959500,12.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,585000,745.0
1,ジーエヌアイグループ,2160,医薬品,827700,1207.0
2,ウェッジホールディングス,2388,その他金融業,8651200,155.0
3,フルッタフルッタ,2586,食料品,1730800,79.0
5,海帆,3133,小売業,2402100,560.0
10,Ａｉｍｉｎｇ,3911,情報・通信業,1630900,465.0
11,ティアンドエス,4055,情報・通信業,518600,2825.0
12,プレイド,4165,情報・通信業,1207100,869.0
15,エクサウィザーズ,4259,情報・通信業,1373500,496.0
20,オンコセラピー・サイエンス,4564,医薬品,4623000,37.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')